In [2]:
%load_ext autoreload
%autoreload 2

import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def get_metrics(datasets, experiment_name, metrics_list):
    """Given a list of datasets, compute the average metrics for each dataset."""
    metrics = [[] for _ in metrics_list]
    for dataset in datasets:
        filename = f"../renders-postprocessed/{dataset}/{dataset}---{experiment_name}/{dataset}---{experiment_name}.json"
        with open(filename, "r") as f:
            metrics_dict = json.load(f)
            for idx, metric_name in enumerate(metrics_list):
                metrics[idx].append(metrics_dict[metric_name])
    # get the mean
    metrics = [sum(metric) / len(metric) for metric in metrics]
    return metrics

In [4]:
TEMPLATE = \
"""
\\begin{table*}[]
    \\centering
    \\begin{tabular}{l|lll|ll|lllll|l} % the number of columns needed
        \\toprule
        COLUMNS \\\\
        \\midrule
        EXPERIMENT_LINES
        \\bottomrule
    \\end{tabular}
    \\caption{\\textbf{CAPTION_TITLE.} CAPTION_TEXT}
    \\label{tab:TABLE_LABEL}
\\end{table*}
"""

In [5]:
COLUMNS = "\n & PSNR $\\uparrow$ & SSIM $\\uparrow$ & LPIPS $\\downarrow$ & Depth $\\downarrow$ & Disp. $\\downarrow$ & Mean $^{\\circ}$ $\\downarrow$ & Median $^{\\circ}$ $\\downarrow$ & \\% $30^{\\circ}$ $\\uparrow$ & Coverage $\\uparrow$"
METRICS_LIST = ["psnr", "ssim", "lpips", "depth", "disparity", "normals", "normals_median", "normals_30", "coverage"]
SIGFIGS = [2, 4, 4, 2, 2, 2, 2, 2, 2]
assert len(METRICS_LIST) == len(SIGFIGS)
# methods are the columns
# here we map the title in the column to the the method name (which we can grab from folder names)
METHODS = [
    ("Nerfacto Pseudo GT   ", "nerfacto---pseudo-gt"),
    ("Nerfacto             ", "nerfacto"),
  #  (" + Visibility Loss   ", "nerfacto-visibility"),
  #  (" + Vis + Sparsity    ", "nerfacto-visibility-sparsity"),
  #  (" + Vis + TV          ", "nerfacto-visibility-TV"),
  #  (" + Vis + RegNeRF     ", "nerfacto-visibility-regnerf"),
    (" + Vis + DSDS (Ours) ", "nerfacto-visibility-cube"),
]
# uncomment this for ablations
# METHODS = [
#     ("Nerfacto GT", "nerfacto---pseudo-gt"),
#     ("Nerfacto          ", "nerfacto"),
#     (" + Visibility Loss", "nerfacto-visibility"),
#     (" + Vis + RegNeRF  ", "nerfacto-visibility-regnerf"),
#     (" + Vis + Sparsity ", "nerfacto-visibility-sparsity"),
#     (" + Vis + TV       ", "nerfacto-visibility-TV"),
#     (" + Vis + Cube     ", "nerfacto-visibility-cube"),
#     # sampling
#     (" + Vis + Cube (sampling-densities)    ", "nerfacto-frustum-cube-sampling-densities"),
#     (" + Vis + Cube (sampling-uniform)     ", "nerfacto-frustum-cube-sampling-uniform"),
#     # activation
#     (" + Vis + Cube (activation-sigmoid)     ", "nerfacto-frustum-cube-activation-sigmoid"),
#     (" + Vis + Cube (activation-clamp)     ", "nerfacto-frustum-cube-activation-clamp"),
#     # cubesize
#     (" + Vis + Cube (cubescale-10-20)     ", "nerfacto-frustum-cube-cubescale-10-20"),
#     (" + Vis + Cube (cubescale-01-20)    ", "nerfacto-frustum-cube-cubescale-01-20"),
# ]
DATASETS = [
    "aloe",
    "art",
    "car",
    "century",
    "flowers",
    "garbage",
    "picnic",
    "pikachu",
    "pipe",
    "plant",
    "roses",
    "table",
]
# DATASETS = ['aloe']
# uncomment this for ablations
#DATASETS = ['garbage']

def print_datasets(datasets, caption_title="caption title", TABLE_LABEL="label", caption_text="caption text"):
    #metrics = get_metrics(datasets, "nerfacto-visibility-sparsity", METRICS_LIST)
    #metrics = [str(f"{metric:.{sigfigs}f}") for metric, sigfigs in zip(metrics, SIGFIGS)]

    # create the experiment lines
    EXPERIMENT_LINES = ["\n"]
    for idx, (method_name, experiment_name) in enumerate(METHODS):
        if idx == 1:
            line_string = f"\\midrule \n"
            EXPERIMENT_LINES.append(line_string)
        metrics = get_metrics(datasets, experiment_name, METRICS_LIST)
        metrics = [str(f"{metric:.{sigfigs}f}") for metric, sigfigs in zip(metrics, SIGFIGS)]
        line_string = f"{method_name} & {' & '.join(metrics)} \\\\ \n"
        EXPERIMENT_LINES.append(line_string)

    s = TEMPLATE.replace("COLUMNS", COLUMNS)
    s = s.replace("EXPERIMENT_LINES", " ".join(EXPERIMENT_LINES))
    s = s.replace("CAPTION_TITLE", caption_title)
    s = s.replace("CAPTION_TEXT", caption_text)
    s = s.replace("TABLE_LABEL", TABLE_LABEL)
    print(s)

# for the average of all the datasets
print_datasets(DATASETS, caption_title="``Average\" capture quantitative evaluation", TABLE_LABEL="average")

print("---" * 10)

# for all the datasets individually
for i in range(len(DATASETS)):
    caption_title = f"""``{DATASETS[i]}\"""" + " capture quantitative evaluation"
    caption_text = """Per scene results that compare Nerfacto with baselines (that uses geometric handcrafted regularizers) and Nerfbusters (that uses a data-driven local 3D prior). """ + \
        """We highlight that all baselines use the proposed visibility loss. \\cref{sec:evaluation_procedure} describes metrics in more detail, $\\uparrow$/$\\downarrow$ indicates if higher/lower is better."""
    print_datasets(DATASETS[i : i + 1], caption_title=caption_title, TABLE_LABEL=DATASETS[i], caption_text=caption_text)



\begin{table*}[]
    \centering
    \begin{tabular}{l|lll|ll|lllll|l} % the number of columns needed
        \toprule
        
 & PSNR $\uparrow$ & SSIM $\uparrow$ & LPIPS $\downarrow$ & Depth $\downarrow$ & Disp. $\downarrow$ & Mean $^{\circ}$ $\downarrow$ & Median $^{\circ}$ $\downarrow$ & \% $30^{\circ}$ $\uparrow$ & Coverage $\uparrow$ \\
        \midrule
        
 Nerfacto Pseudo GT    & 24.81 & 0.8086 & 0.1485 & 0.00 & 0.00 & 0.01 & 0.00 & 1.00 & 0.89 \\ 
 \midrule 
 Nerfacto              & 17.23 & 0.5308 & 0.3736 & 232.36 & 0.53 & 60.70 & 54.79 & 0.26 & 0.87 \\ 
  + Vis + DSDS (Ours)  & 17.92 & 0.6071 & 0.2516 & 142.99 & 0.09 & 56.33 & 49.86 & 0.29 & 0.61 \\ 

        \bottomrule
    \end{tabular}
    \caption{\textbf{``Average" capture quantitative evaluation.} caption text}
    \label{tab:average}
\end{table*}

------------------------------

\begin{table*}[]
    \centering
    \begin{tabular}{l|lll|ll|lllll|l} % the number of columns needed
        \toprule
        
 & PSNR 